In [1]:
import warnings
warnings.simplefilter(action='ignore', category=UserWarning)

import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import gzip
from datetime import datetime, timedelta
from statistics import mean, median
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
import seaborn as sns
import matplotlib.pyplot as plt

import tensorflow
import tensorflow.keras as tf
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Input, Dense, LeakyReLU, BatchNormalization, ReLU, LSTM, Conv1D, Conv2D
from tensorflow.keras.activations import sigmoid, tanh
from tensorflow.keras.utils import to_categorical

from sklearn.preprocessing import MinMaxScaler

from tqdm import tqdm
import csv
import random
import math

from sklearn.metrics import accuracy_score as accuracy
from sklearn.metrics import precision_score as precision
from sklearn.metrics import recall_score as recall
from sklearn.metrics import f1_score as f1

import pickle

In [2]:
def extract_model(filename):
    model = tf.models.load_model(filename)
    return model
    
def retrieve_data(filename):
    df = pd.read_csv(filename, index_col=0)
    df["Date"] = pd.to_datetime(df["Date"])
    return df

def create_classification_data(df, lookback, column):
    rows = []
    columns = ['Date', column] # Date and SP500_relative_change_perc_1 from t-0 are added first as target variables 
    
    # create column names based on original with the addition of t-i where i is lookback
    for i in range(1, lookback + 1): # starts at 1 since we do not want t-0 variables apart from 'Date' and 'SP500_relative_change_perc_1'
        new_columns = df.columns.tolist()[1:] # starts at 1 to exclude 'Date' column
        for x in range(len(new_columns)):
            new_columns[x] = new_columns[x] + "_t-" + str(i)
        columns = columns + new_columns
    
    # create lookback data
    for i, row in enumerate(df.iterrows()):
        if i > lookback: # lookback cannot be determined for earlier rows
            new_row = [row[1][0], row[1][1]] # add target 'Date' and 'SP500_relative_change_perc_1 '
            for x in range(1, lookback + 1): # starts at 1 since we do not want t-0 variables apart from 'Date' and 'SP500_relative_change_perc_1'
                add_row = df.iloc[i - x].tolist()[1:] # starts at 1 to exclude 'Date' column
                new_row = new_row + add_row
            rows.append(new_row)
    df2 = pd.DataFrame(rows)
    df2.columns = columns
    return df2

def create_train(df, year_val, year_test, column, perc_train=None):
    # assumes years_train < year_val < year_test
    df["Date"] = pd.to_datetime(df["Date"])

    val = df[df['Date'].dt.year == year_val]
    test = df[df['Date'].dt.year == year_test]
    train = df[df['Date'].dt.year < year_val]
    y_train = train[column]
    x_train = train
    return x_train


def create_val(df, year_val, year_test, column, perc_train=None):
    # assumes years_train < year_val < year_test
    df["Date"] = pd.to_datetime(df["Date"])
    val = df[df['Date'].dt.year == year_val]
    test = df[df['Date'].dt.year == year_test]
    train = df[df['Date'].dt.year < year_val]
    y_val = val[column]
    x_val = val
#     display(x_val)
    return x_val

def create_test(df, year_val, year_test, column, perc_train=None):
    print("test", len(df))
    # assumes years_train < year_val < year_test
    df["Date"] = pd.to_datetime(df["Date"])

    val = df[df['Date'].dt.year == year_val]
    test = df[df['Date'].dt.year == year_test]
    train = df[df['Date'].dt.year < year_val]
    y_test = test[column]
    x_test = test
    return x_test

def create_full(df, year_val, year_test, column, perc_train=None):
    df["Date"] = pd.to_datetime(df["Date"])
    y = df[column]
    x = df
    
    return x

In [3]:
"""
    Assumptions:
        You can invest precisely at market open until (exactly) market close
        Investment amounts can be up to 10 decimal points
        Geen commission
        Geen exchange rates
        Trader can go long or short

    Requirements:
        Make sure to consider lookback of each model
        Add recent accuracy (yesterday (0 or 100%), last week, last 2 weeks) as additional features for final NN
"""


'\n    Assumptions:\n        You can invest precisely at market open until (exactly) market close\n        Investment amounts can be up to 10 decimal points\n        Geen commission\n        Geen exchange rates\n        Trader can go long or short\n\n    Requirements:\n        Make sure to consider lookback of each model\n        Add recent accuracy (yesterday (0 or 100%), last week, last 2 weeks) as additional features for final NN\n'

In [4]:
def extract_data(instruments, data, mode):
    year_val = 2018
    year_test = 2019
    for symbol in instruments:
        column = symbol + "_relative_change_perc_1"
        ndir = symbol + "_dir"
        nmag = symbol + "_mag"

        data_dir = data[ndir]
        data_mag = data[nmag]
        if mode == "train":
            data[ndir] = create_train(data_dir, year_val, year_test, column)
            data[nmag] = create_train(data_mag, year_val, year_test, column)
        if mode == "val":
            data[ndir] = create_val(data_dir, year_val, year_test, column)
            data[nmag] = create_val(data_mag, year_val, year_test, column)
        if mode == "test":
            data[ndir] = create_test(data_dir, year_val, year_test, column)
            data[nmag] = create_test(data_mag, year_val, year_test, column)
        if mode == "full":
            data[ndir] = create_full(data_dir, year_val, year_test, column)
            data[nmag] = create_full(data_mag, year_val, year_test, column)
    return data

def init_actions(granularity=0.2):
    possible = []
    for i in np.linspace(0, 1, int(1 / granularity + 1)).tolist():
        possible.append(round(i,1))    
    actions = []
    for x in possible:
        for y in possible:
            for z in possible:
                if x + y + z == 1:
                    actions.append([x, y, z])
#     print(f"Number of possible actions: {len(actions)}")
    actions = [[0.333, 0.333, 0.333]]
    return actions

def init_Q(actions):


    Q = {}
    pdir = np.array([0,1])
    pmag = np.array([0,1])

    for i0 in pdir:
        for i1 in pmag:
            for i2 in pdir:
                for i3 in pmag:
                    for i4 in pdir:
                        for i5 in pmag:
                            sub = [i0, i1, i2, i3, i4, i5]
#                             print(sub)
                            Q[repr(sub)] = [0] * len(actions)
    return Q

def choose_action(state, epsilon, actions, epsilon_greedy=True):                   
    action = 0
    return action

def predict_individual(date, x, model, column):
    x = x[x['Date'] == date]
    if x.empty:
#         print(date, "empty")
        return 0
#     x = x.drop(['Date'], axis=1)
#     x = x.drop([column], axis=1)
#     x = np.asarray(x)
#     x = x.reshape((x.shape[0], 1, x.shape[1]))
#     pred = int(model.predict(x, batch_size=1).round())
    pred = x[column].iloc[0]
    return pred

def predict(date, instruments, data, models):
    state = []
    for symbol in instruments:
        column = symbol + "_relative_change_perc_1"
        ndir = symbol + "_dir"
        
        model_dir = models[ndir]
        data_dir = data[ndir]
        
        # direction prediction
#         pred = predict_individual(date, data_dir, model_dir, column)
        pred = 1
        state.append(pred)
        
    return state

def translate_action(action_index, actions):
    return actions[action_index]
    
def get_individual_reward(date, x, direction, action, column, capital, risk):
    x = x[x['Date'] == date]
    if x.empty:
        return 0

    relative_change = x[column].iloc[0]
    if np.sign(relative_change) == np.sign(direction):
        relative_change = abs(relative_change)
    else:
        relative_change = -1 * abs(relative_change)
    invested_capital = capital * risk
    profit = relative_change * action * invested_capital
    
    return profit
    

def get_reward(date, data, state, action, capital, risk):
    profit = 0
    for i, symbol in enumerate(instruments):
        column = symbol + "_relative_change_perc_1"
        ndir = symbol + "_dir"
        data_dir = data[ndir]
        
        direction = 1
            
        profit += get_individual_reward(date, data_dir, direction, action[i], column, capital, risk)
    return profit

def get_reward_eval(date, data, state, action, capital, risk, metrics):
    profit = 0
    for i, symbol in enumerate(instruments):
        column = symbol + "_relative_change_perc_1"
        ndir = symbol + "_dir"
        data_dir = data[ndir]
    
        direction = 1
            
        individual_reward = get_individual_reward(date, data_dir, direction, action[i], column, capital, risk)
        
        if individual_reward > 0:
            metrics['win_trades'] += 1
            metrics['average_win_trade'] += individual_reward
        elif individual_reward < 0:
            metrics['loss_trades'] += 1
            metrics['average_loss_trade'] += individual_reward
        profit += individual_reward
    return profit, metrics

In [9]:
def evaluate_Q(Q, instruments, data, capital, risk, mode):
    
    capitals = []
#     capitals.append(capital)
    data = extract_data(instruments, data, mode)
    dates = data[instruments[0] + "_dir"]['Date'].tolist()
    peak = capital
    valley = capital
    
    metrics = {
        'start_date': dates[0],
        'end_date': dates[-1],
        'profit' : 0,
        'relative_profit': 0,
        'max_drawdown': 0,
        'rel_max_drawdown': 0,
        'max_run_up': 0,
        'rel_max_run_up': 0,
        
        'trades': 0,
        'win_trades': 0,
        'loss_trades': 0,
        'relative_win_trades': 0,
        
        'days_traded': len(dates),
        'win_days': 0,
        'loss_days': 0,
        'relative_win_days': 0,
        
        'average_trade': 0,
        'average_win_trade': 0,
        'average_loss_trade': 0,
        'ratio_wl_trade': 0,
        
        'average_day': 0,
        'average_win_day': 0, 
        'average_loss_day': 0,
        'ratio_wl_day': 0,
        'max_consecutive_wins': 0 
    }
    
    
    actions = init_actions()
    consec_wins = 0
    for d in tqdm(dates, total=len(dates)):
        date = pd.to_datetime(d)
        state = predict(date, instruments, data, models)
        action_index = choose_action(state, epsilon, actions, False)
        action = translate_action(action_index, actions)
        for a in action:
            if a > 0:
                metrics['trades'] += 1
        reward, metrics = get_reward_eval(date, data, state, action, capital, risk, metrics)

        metrics['profit'] += reward
        if reward >= 0:
            metrics['win_days'] += 1 
            metrics['average_win_day'] += reward
            consec_wins += 1
            if consec_wins > metrics['max_consecutive_wins']:
                metrics['max_consecutive_wins'] = consec_wins
        else:
            consec_wins = 0
            metrics['loss_days'] += 1   
            metrics['average_loss_day'] += reward
            
        capital += reward
        capitals.append(capital)
        
        if capital > peak:
            peak = capital
        else:
            if peak - capital > metrics['max_drawdown']:
                metrics['max_drawdown'] = peak - capital
            if (peak - capital) / peak > metrics['rel_max_drawdown']:
                metrics['rel_max_drawdown'] = (peak - capital) / peak
            
        if capital < valley:
            valley = capital
        else:
            if capital - valley > metrics['max_run_up']:
                metrics['max_run_up'] = capital - valley
            if (capital - valley) / valley > metrics['rel_max_run_up']:
                metrics['rel_max_run_up'] = (capital - valley) / valley
        
    metrics['relative_profit'] = metrics['profit'] / capitals[0]
    if metrics['trades'] > 0:
        metrics['relative_win_trades'] = metrics['win_trades'] / metrics['trades']
    else:
        metrics['relative_win_trades'] = 0
        
    if metrics['days_traded'] > 0:
        metrics['relative_win_days'] = metrics['win_days'] / metrics['days_traded']
    else:
        metrics['relative_win_days'] = 0
    
    metrics['average_day'] = metrics['average_day'] / len(dates)
    if metrics['win_days'] > 0:
        metrics['average_win_day'] = metrics['average_win_day'] / metrics['win_days']
    else:
        metrics['average_win_day'] = 0
        
    if metrics['loss_days'] > 0:
        metrics['average_loss_day'] = abs(metrics['average_loss_day'] / metrics['loss_days'])
    else:
        metrics['average_loss_day'] = 0
    
    if metrics['average_loss_day'] > 0:
        metrics['ratio_wl_day'] = metrics['average_win_day'] / metrics['average_loss_day']
    else:
        metrics['ratio_wl_day'] = 0
    
    if metrics['trades'] > 0:
        metrics['average_trade'] = metrics['average_trade'] / metrics['trades']
    else:
        metrics['average_trade'] = 0
    
    if metrics['win_trades'] > 0:
        metrics['average_win_trade'] = metrics['average_win_trade'] / metrics['win_trades']
    else:
        metrics['average_win_trade'] = 0
        
    if metrics['loss_trades'] > 0:
        metrics['average_loss_trade'] = abs(metrics['average_loss_trade'] / metrics['loss_trades'])
    else:
        metrics['average_loss_trade'] = 0
    
    if metrics['average_loss_trade'] > 0:
        metrics['ratio_wl_trade'] = metrics['average_win_trade'] / metrics['average_loss_trade']
    else:
        metrics['ratio_wl_trade'] = 0
    return metrics, capitals, dates

def run(alpha, gamma, epsilon, decay, episodes, instruments, data, models, capital, risk, mode):
    actions = init_actions()
#     Q = init_Q(actions)
    total_rewards = []
    mean_diffs = []
    
    data = extract_data(instruments, data, mode)
#     display(data)
    dates = data[instruments[0] + "_dir"]['Date'].tolist()
    
    for i in range(episodes):
        print(f"Executing episode {i + 1} / {episodes}")
        total_reward = 0
        diffs = []
#         date = dates[0]
#         dates = dates[1:]
#         # DOES NOT WORK IF DATE IS NOT IN DATASET
#         state = predict(date, instruments, data, models)
#         action_index = choose_action(Q, state, epsilon, actions)
#         action = translate_action(action_index, actions)
#         print(f"State {state} Action {action}")
        
        for d in tqdm(dates, total=len(dates)):
            date = pd.to_datetime(d)

            ## CREATIEVE IMPLEMENTATIE
            state = predict(date, instruments, data, models)
            action_index = choose_action(Q, state, epsilon, actions)
            action = translate_action(action_index, actions)
            reward = get_reward(date, data, state, action, capital, risk)
#             print(f"State {state} Action {action} Reward {reward}")
            
#             Q_old = Q[str(state)][action_index]
#             Q[str(state)][action_index] += alpha * reward
# #             Q[str(state)][action_index] += alpha * (next_reward + gamma * Q[str(next_state)][next_action_index] - Q[str(state)][action_index])
#             Q_new = Q[str(state)][action_index] 
#             diffs.append(abs(Q_new - Q_old))
            
            total_reward += reward
            capital += reward
        
#         Q_file = f"Models/Q/Q{i}.pickle"
        with open(Q_file, 'wb') as handle:
            pickle.dump(Q, handle, protocol=pickle.HIGHEST_PROTOCOL)
        epsilon = epsilon * decay
#         alpha = alpha * decay
        mean_diffs.append(sum(diffs) / len(diffs))
        total_rewards.append(total_reward)
    return total_rewards, mean_diffs

In [6]:
def init_models_data():
    instruments = ['SP500', 'NASDAQ', 'US30']

    model_mag_sp500 = extract_model("Models/SP500_LSTM_large-small_model")
    model_mag_us30 = extract_model("Models/US30_LSTM_large-small_model")
    model_mag_nasdaq = extract_model("Models/NASDAQ_LSTM_large-small_model")

    model_dir_sp500 = extract_model("Models/SP500_NN_up-down_model")
    model_dir_us30 = extract_model("Models/US30_NN_up-down_model")
    model_dir_nasdaq = extract_model("Models/NASDAQ_NN_up-down_model")

    models = {
        'SP500_mag': model_mag_sp500,
        'US30_mag': model_mag_us30,
        'NASDAQ_mag': model_mag_nasdaq,
        'SP500_dir': model_dir_sp500,
        'US30_dir': model_dir_us30,
        'NASDAQ_dir': model_dir_nasdaq
    }

    data_mag_sp500 = create_classification_data(retrieve_data("Dataset v3/SP500_reduced_data_20220425.csv"), 2, 'SP500_relative_change_perc_1')
    data_mag_us30 = create_classification_data(retrieve_data("Dataset v3/US30_reduced_data_20220425.csv"), 12, 'US30_relative_change_perc_1')
    data_mag_nasdaq = create_classification_data(retrieve_data("Dataset v3/NASDAQ_reduced_data_20220425.csv"), 18, 'NASDAQ_relative_change_perc_1')

    data_dir_sp500 = create_classification_data(retrieve_data("Dataset v3/SP500_reduced_data_20220425.csv"), 12, 'SP500_relative_change_perc_1')
    data_dir_us30 = create_classification_data(retrieve_data("Dataset v3/US30_reduced_data_20220425.csv"), 10, 'US30_relative_change_perc_1')
    data_dir_nasdaq = create_classification_data(retrieve_data("Dataset v3/NASDAQ_reduced_data_20220425.csv"), 10, 'NASDAQ_relative_change_perc_1')

    data = {
        'SP500_mag': data_mag_sp500,
        'US30_mag': data_mag_us30,
        'NASDAQ_mag': data_mag_nasdaq,
        'SP500_dir': data_dir_sp500,
        'US30_dir': data_dir_us30,
        'NASDAQ_dir': data_dir_nasdaq
    }
    return instruments, data, models

In [7]:
alpha = 0.1
gamma = 0.1
epsilon = 0.9
decay = 0.85
episodes = 50

capital = 10000
risk = 0.05 # capital risked per day

instruments, data, models = init_models_data()

mode = 'train'
# Q, total_rewards, mean_diffs = run(alpha, gamma, epsilon, decay, episodes, instruments, data, models, capital, risk, mode)
# profit, risk = evaluate_Q(Q)
# profit = evaluate_Q(Q)

2022-06-10 17:02:53.953356: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-06-10 17:02:53.953596: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [10]:
def plot_capitals(dates, capitals):
    fig1 = make_subplots(rows=1, cols=1, specs=[[{'type':'xy'}]])
    
    fig1.add_trace(go.Scatter(x=dates, y=capitals, mode="lines"), row=1, col=1)
#     fig1.add_vline(x=2.5, line_width=3, line_dash="dash", line_color="green")

    fig1.update_layout(
        title = f'Compounded Capital over Time', 
        xaxis1 = dict(title_text = 'Day'),
        yaxis1 = dict(title_text = 'Capital')
    )
    fig1.show()
    
capital = 1000000
risk = 1


actions = init_actions()
Q = init_Q(actions)

print("Evaluating...")

instruments, data, models = init_models_data()
mode = 'train'
metrics, capitals, dates = evaluate_Q(Q, instruments, data, capital, risk, mode)
# print(f"Profit in evaluation: {profit}")
plot_capitals(dates, capitals)
display(metrics)
df = pd.DataFrame({'dates': dates, 'capitals': capitals})
df.to_csv("Portfolio Output/maj_equal_weight_train_100.csv")

instruments, data, models = init_models_data()
mode = 'val'
metrics, capitals, dates = evaluate_Q(Q, instruments, data, capital, risk, mode)
# print(f"Profit in evaluation: {profit}")
plot_capitals(dates, capitals)
display(metrics)
df = pd.DataFrame({'dates': dates, 'capitals': capitals})
df.to_csv("Portfolio Output/maj_equal_weight_val_100.csv")

instruments, data, models = init_models_data()
mode = 'test'
metrics, capitals, dates = evaluate_Q(Q, instruments, data, capital, risk, mode)
# print(f"Profit in evaluation: {profit}")
plot_capitals(dates, capitals)
display(metrics)
df = pd.DataFrame({'dates': dates, 'capitals': capitals})
df.to_csv("Portfolio Output/maj_equal_weight_test_100.csv")

instruments, data, models = init_models_data()
mode = 'full'
metrics, capitals, dates = evaluate_Q(Q, instruments, data, capital, risk, mode)
# print(f"Profit in evaluation: {profit}")
plot_capitals(dates, capitals)
display(metrics)
df = pd.DataFrame({'dates': dates, 'capitals': capitals})
df.to_csv("Portfolio Output/maj_equal_weight_full_100.csv")

Evaluating...


100%|██████████████████████████████████████| 2128/2128 [00:05<00:00, 380.12it/s]


{'start_date': Timestamp('2009-07-21 00:00:00'),
 'end_date': Timestamp('2017-12-29 00:00:00'),
 'profit': 1273091.2777667905,
 'relative_profit': 1.2689575412574876,
 'max_drawdown': 267711.03509126394,
 'rel_max_drawdown': 0.151582741920661,
 'max_run_up': 1306378.7633693432,
 'rel_max_run_up': 1.3063787633693433,
 'trades': 6384,
 'win_trades': 3504,
 'loss_trades': 2878,
 'relative_win_trades': 0.5488721804511278,
 'days_traded': 2128,
 'win_days': 1182,
 'loss_days': 946,
 'relative_win_days': 0.5554511278195489,
 'average_trade': 0.0,
 'average_win_trade': 3047.132397109061,
 'average_loss_trade': 3267.5679783541927,
 'ratio_wl_trade': 0.9325383334928626,
 'average_day': 0.0,
 'average_win_day': 8442.68743351954,
 'average_loss_day': 9203.134533460143,
 'ratio_wl_day': 0.9173708591159002,
 'max_consecutive_wins': 12}

100%|████████████████████████████████████████| 251/251 [00:00<00:00, 439.88it/s]


{'start_date': Timestamp('2018-01-02 00:00:00'),
 'end_date': Timestamp('2018-12-31 00:00:00'),
 'profit': -138741.77532427156,
 'relative_profit': -0.1380488256003328,
 'max_drawdown': 235151.55244187836,
 'rel_max_drawdown': 0.22514759844348198,
 'max_run_up': 93212.74031767307,
 'rel_max_run_up': 0.10062457942566237,
 'trades': 753,
 'win_trades': 393,
 'loss_trades': 360,
 'relative_win_trades': 0.5219123505976095,
 'days_traded': 251,
 'win_days': 131,
 'loss_days': 120,
 'relative_win_days': 0.5219123505976095,
 'average_trade': 0.0,
 'average_win_trade': 2055.783551763038,
 'average_loss_trade': 2629.6241976865117,
 'ratio_wl_trade': 0.7817784585233408,
 'average_day': 0.0,
 'average_win_day': 5859.190611719326,
 'average_loss_day': 7552.46454549586,
 'ratio_wl_day': 0.7757984928527246,
 'max_consecutive_wins': 9}

...

test 2631
test 2641
test 2633
test 2625
test 2633
test 2631


100%|████████████████████████████████████████| 252/252 [00:00<00:00, 427.91it/s]


{'start_date': Timestamp('2019-01-02 00:00:00'),
 'end_date': Timestamp('2019-12-31 00:00:00'),
 'profit': 169432.39765567868,
 'relative_profit': 0.16664308046421228,
 'max_drawdown': 83997.51627644151,
 'rel_max_drawdown': 0.07093386428129683,
 'max_run_up': 186806.7810318116,
 'rel_max_run_up': 0.18730128138246355,
 'trades': 756,
 'win_trades': 428,
 'loss_trades': 328,
 'relative_win_trades': 0.5661375661375662,
 'days_traded': 252,
 'win_days': 149,
 'loss_days': 103,
 'relative_win_days': 0.5912698412698413,
 'average_trade': 0.0,
 'average_win_trade': 1740.578184611309,
 'average_loss_trade': 1754.6800773108578,
 'ratio_wl_trade': 0.9919632684716175,
 'average_day': 0.0,
 'average_win_day': 4847.714664618327,
 'average_loss_day': 5367.738712353903,
 'ratio_wl_day': 0.9031204617805381,
 'max_consecutive_wins': 11}

100%|██████████████████████████████████████| 2631/2631 [00:08<00:00, 323.78it/s]


{'start_date': Timestamp('2009-07-21 00:00:00'),
 'end_date': Timestamp('2019-12-31 00:00:00'),
 'profit': 1289419.5077026347,
 'relative_profit': 1.285232753313627,
 'max_drawdown': 534520.9428089564,
 'rel_max_drawdown': 0.22514759844348284,
 'max_run_up': 1374091.247271879,
 'rel_max_run_up': 1.3740912472718791,
 'trades': 7893,
 'win_trades': 4325,
 'loss_trades': 3566,
 'relative_win_trades': 0.5479538831876346,
 'days_traded': 2631,
 'win_days': 1462,
 'loss_days': 1169,
 'relative_win_days': 0.5556822500950209,
 'average_trade': 0.0,
 'average_win_trade': 3230.53667648548,
 'average_loss_trade': 3556.5484066452796,
 'ratio_wl_trade': 0.908334797425881,
 'average_day': 0.0,
 'average_win_day': 8986.358047133202,
 'average_loss_day': 10135.702273059112,
 'ratio_wl_day': 0.8866043817229233,
 'max_consecutive_wins': 12}